In [1]:
import multiprocessing
import time
from sentientresearchagent.framework_entry import ProfiledSentientAgent
import traceback

def run_agent_execution(query: str):
    """
    A target function for each process to execute a query.
    It creates its own agent instance in its own memory space.
    """
    process_name = multiprocessing.current_process().name
    
    try:
        # Each process creates its own agent instance. This guarantees
        # that all state, including the agent registry, is completely isolated.
        print(f"🤖 [{process_name}] Creating a dedicated agent instance in a new process...")
        agent = ProfiledSentientAgent.create_with_profile(
            profile_name="general_agent",
            enable_hitl_override=False,
            hitl_root_plan_only_override=False,
            hitl_root_plan_only=False,
            max_planning_depth=3,
            max_concurrent_tasks=10,
        )
        
        print(f"🚀 [{process_name}] Starting execution for query: '{query}'")
        start_time = time.time()

        result = agent.execute(goal=query)

        end_time = time.time()
        print(f"✅ [{process_name}] Finished in {end_time - start_time:.2f} seconds.")

        # In multiprocessing, returning complex objects can be tricky.
        # For this example, we'll just print a summary.
        final_output = result.get('final_output', 'No output generated.')
        print(f"📄 [{process_name}] Result for '{query}':\n   {final_output[:200].replace('\n', '\n   ')}...\n")

    except Exception as e:
        end_time = time.time()
        print(f"❌ [{process_name}] An error occurred: {e}")
        traceback.print_exc()

In [2]:
# A list of independent queries to be executed in parallel.
queries = [
    "What are the key benefits of using asyncio in Python?",
    "Provide a brief history of the city of Rome.",
    "Explain the concept of quantum entanglement in simple terms.",
]

processes = []

print("🏁 Starting agent executions, each in its own isolated process...")
for i, query in enumerate(queries):
    # Create a Process for each query.
    process = multiprocessing.Process(
        target=run_agent_execution,
        args=(query,),
        name=f"AgentProcess-{i+1}"
    )
    processes.append(process)
    process.start() # Start the new process

# Wait for all processes to complete their execution
for process in processes:
    process.join()

🏁 Starting agent executions, each in its own isolated process...


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/salahalzubi/.pyenv/versions/3.12.9/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/salahalzubi/.pyenv/versions/3.12.9/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^    ^exitcode = _main(fd, parent_sentinel)^
^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/salahalzubi/.pyenv/versions/3.12.9/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/salahalzubi/.pyenv/versions/3.12.9/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    self = reduction.pickle.load(from_parent)AttributeError
: Can't get attribute 'run_agent_execution' on <module '__main__' (<class '_fr